In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import hail as hl
hl.init(
    tmp_dir='/net/ascratch/people/plggosborcz/gosborcz-hail',
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '30G'},
    default_reference='GRCh38') 

2023-01-12 10:12:03.544 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0079:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.105-acd89e80c345
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/analysis/oligogenic-model/hail-20230112-1012-0.2.105-acd89e80c345.log


In [3]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat
from collections import Counter
    
import bokeh.palettes

from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

Loading BokehJS ...

#### important note: file paths in this notebook are outdated 

## Load SKAT functions

In [5]:
def remove_sex_chrom(mtx):
    mtx = mtx.filter_rows(mtx.locus.contig != "chrY")
    return(mtx)


def run_pca(mtx, mtx_subset):
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(mtx_subset.GT)
    mtx = mtx.annotate_cols(scores = pcs[mtx.s].scores)

    return(mtx)

def run_skat_log(mtx, gene_list, pcs):
    
    mtx = mtx.filter_rows(hl.any(lambda x: hl.literal(gene_list).contains(x), mtx.nearest_genes_20kb))
    mtx = mtx.filter_rows(hl.agg.any(mtx.GT.is_non_ref()))
    mtx = mtx.explode_rows(mtx.nearest_genes_20kb)
    mtx = mtx.filter_rows(hl.literal(gene_list).contains(mtx.nearest_genes_20kb))
    
    
    scores = [mtx.scores[x] for x in list(range(pcs))]
                          
    
    skat_table = hl.skat(
                         key_expr=mtx.nearest_genes_20kb,
                         weight_expr=mtx.cadd,
                         y=mtx.category,
                         x=mtx.GT.n_alt_alleles(),
                         covariates=[1] + scores,
                         max_size = 2500,
                         logistic = True)
    
    genes_result = skat_table.filter(skat_table.p_value < 0.05/len(gene_list)).id.collect() 

    skat_table.filter(skat_table.p_value < 0.002).show(20)

    skat_table = skat_table.annotate(label = hl.literal(genes).contains(skat_table.id))

    qq_plot = hl.plot.qq(skat_table.p_value,
                                         label = skat_table.label,
                                         n_divisions = len(gene_list))
    show(qq_plot)
    
    return(skat_table, genes_result, qq_plot)


def full_skat_log(mtx, mtx_subset, gene_list, pcs):
    
    mtx = remove_sex_chrom(mtx)
    mtx_subset = remove_sex_chrom(mtx_subset)
  
    mtx = run_pca(mtx, mtx_subset) #this matrix will be returned, so I can do SKAT with other list and parameters
    skat_table, genes_result, qq_plot = run_skat_log(mtx, gene_list, pcs)
    
    return(mtx, skat_table, genes_result, qq_plot)

In [11]:
def test_model(geneset_name):  
    for c_idx, c in enumerate(cadds):

        test_asignment = np.zeros((len(top_genes), 145)) 
        mt_skat_log = hl.read_matrix_table(
            '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt_for_skat_'+str(c)+'.mt'
        )
        mt_test_skat_log = hl.read_matrix_table(
            '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt_test_'+str(c)+'.mt'
        )
      
        for rows, n in enumerate(top_genes):

            mt_skat_log_gene = mt_skat_log.filter_rows(mt_skat_log.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
        
            mt_test_skat_log_gene = mt_test_skat_log.filter_rows(mt_test_skat_log.nearest_genes_20kb.contains(n)) # to ma rodziny

            mt_skat_log_gene = mt_skat_log_gene.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log_gene.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
            non_refs = mt_skat_log_gene.non_refs.collect()
                                                
            mt_test_skat_log_gene = mt_test_skat_log_gene.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log_gene.GT.is_non_ref())) #count variants per sample, prepare also test matrix
            non_refs_test = mt_test_skat_log_gene.non_refs.collect()

            variants_controls = np.mean(np.array(non_refs)[np.invert(categories)])
                
            test_asignment[rows] = (non_refs_test - variants_controls)
            
            print(test_asignment.shape)
                
        for s in sets:
        
            test_asignment_subset = test_asignment[range(5-s,5), :]
            
            print(test_asignment_subset.shape)
                
            test_asignment_subset = np.sum(test_asignment_subset, axis = 0)
                
            false_pos = []
            true_pos = []
            
            for x in np.linspace(-100,100,10000):
            
                false_pos.append(np.sum((test_asignment_subset > x)[np.invert(categories_test)])/53)
                true_pos.append(np.sum((test_asignment_subset > x)[categories_test])/91)
            
            print(np.trapz(false_pos, true_pos))
        
            np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/false_pos'+str(s)+'cadd'+str(c)+'gene'+geneset_name, false_pos)
            np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/true_pos'+str(s)+'cadd'+str(c)+'gene'+geneset_name, true_pos)


In [37]:
def full_model(gene_list, geneset_name, mt_for_model):
    
    skat_table, genes_result, qq_plot = run_skat_log(mt_for_model, gene_list, 8)
    top_genes = skat_table.order_by('p_value').id.take(5)
    test_model(geneset_name)

### SKAT overrepresentation analysis:

In [13]:
genes = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2'] # the other gene next to chadl - 'L3MBTL2' was deleted not to confuse the analysis

allgenes = hl.import_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/external-data/gts-gene-lists/human-genes-with-GO-and-symbols') 
allgenes = allgenes.select('UniProtKB Gene Name symbol')

allgenes = allgenes.filter(allgenes['UniProtKB Gene Name symbol'] != "")
allgenes = allgenes['UniProtKB Gene Name symbol'].collect()

genes_scores = list(set(genes))

2023-01-12 10:18:40.572 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)


In [17]:
mt = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/GTS-gnomad-sex.mt'
)

#### now annotate with new phenotypes (update from the clinician)

In [30]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)

mt = mt.annotate_cols(phenotypes = pheno[mt.s])
mt = mt.filter_cols(mt.s.contains('gnomad') | (mt.phenotypes.family == '.'))
mt = mt.annotate_cols(
    category = hl.if_else(mt.s.contains('gnomad'),
                          False,
                          (mt.phenotypes.heavy_tics == 'YES')
                         )
)

samples = mt.s.collect()

#first 70 gnomad samples are females (see GTS-annotate-cadd-genes)
gnomad_females = samples[40:(40+70)]
mt = mt.transmute_cols(
    phenotypes = mt.phenotypes.annotate(
        sex = hl.if_else(mt.s.contains('gnomad'),
                         hl.if_else(
                             hl.array(gnomad_females).contains(mt.s),
                             'F',
                             'M'),
                         mt.phenotypes['sex'])
    )
)

sexes = mt.phenotypes.sex.collect()

to_keep = samples[0:40] + samples[40:(40+6)] + samples[40+71:40+71+34] #this first 6 gnomad females + first 33 males
mt = mt.filter_cols(hl.array(to_keep).contains(mt.s)) # filter out excessive gnomads
mt = mt.filter_rows(mt.cadd > 0) # this is so I don't do too many tests
mt = mt.naive_coalesce(250)

mt.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-skat.mt'
)

2023-01-12 10:29:55.000 Hail: INFO: Reading table to impute column types
2023-01-12 10:29:55.255 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)
2023-01-12 10:32:04.246 Hail: INFO: wrote matrix table with 6437977 rows and 80 columns in 246 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-skat.mt


### now make a subset (same genotypes as previously)

In [31]:
mt_for_skat = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-skat.mt')
mt_subset = mt_for_skat.sample_rows(0.002)
mt_subset = mt_subset.naive_coalesce(50)
mt_subset.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-skat-subset.mt')

2023-01-12 10:32:12.468 Hail: INFO: wrote matrix table with 13003 rows and 80 columns in 50 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-skat-subset.mt
    Total size: 6.65 MiB
    * Rows/entries: 6.65 MiB
    * Columns: 653.00 B
    * Globals: 11.00 B
    * Smallest partition: 120 rows (52.12 KiB)
    * Largest partition:  286 rows (186.35 KiB)


### run SKAT:

In [35]:
mt_for_skat = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-skat.mt'
)
mt_subset = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-skat-subset.mt'
)

In [ ]:
a = mt_for_skat.phenotypes.heavy_tics.collect()
b = mt_for_skat.phenotypes.add_pheno.collect()
c = mt_for_skat.phenotypes.disease.collect()

mt_for_model, skat_table_log, genes_result_log, qq_plot_log = full_skat_log(mt_for_skat, mt_subset, genes_scores, 8)

#### Prepare matrix table

In [ ]:
skat_table_log.write('/net/archive/groups/plggneuromol/GTS-analysis/data/skat-expanded-may.ht')
mt_for_model.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-for-model-expanded-may.mt')

### Collect the number of variants in control individuals for the classifier

A few iterations over various numbers of genes and CADD score cutt-offs were run

In [ ]:
skat = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/skat-expanded-may.ht')

In [ ]:
plot = hl.plot.qq(skat.p_value)

plot.xaxis.axis_label_text_font_size = "15pt"
plot.xaxis.major_label_text_font_size = "15pt"
plot.yaxis.axis_label_text_font_size = "15pt"
plot.yaxis.major_label_text_font_size = "15pt"

In [ ]:
show(plot)

In [ ]:
skat.order_by('p_value').show(20)

### reimport the matrixtable again and prepare the test dataset


###### reimport the phenotype table again as annotation of WGS_188a has been corrected

In [ ]:
mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/GTS-gnomad-sex.mt')

pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)

mt_test = mt_test.annotate_cols(phenotypes = pheno[mt_test.s])

mt_test = mt_test.annotate_cols(category = hl.if_else(mt_test.s.contains('gnomad'), False, (mt_test.phenotypes.disease == 'YES')))

mt_test = mt_test.filter_cols(mt_test.phenotypes.family == '.', keep = False)
mt_test = mt_test.filter_cols((mt_test.phenotypes.disease == 'YES') | (mt_test.phenotypes.disease == 'NO'))

mt_test = mt_test.filter_rows(mt_test.cadd > 0) 

#mt_test.write('/net/archive/groups/plggneuromol/GTS-analysis/data/mt-test-may.mt')
mt_for_skat = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/mt-for-skat-may.mt')
mt_test = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/mt-test-may.mt')
skat = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/mts/skat-expanded-may.ht')

categories = mt_for_skat.category.collect()
categories_test = mt_test.category.collect()

np.where(np.array(categories_test) == True)[0].shape
np.where(np.array(categories_test) == False)[0].shape

### test the test dataset 

#### how to calculate confusion matrix values

sum(asignment[np.invert(categories)]) #number of false positives

sum(np.invert(asignment)[categories]) # number of false negatives
        
sum(np.invert(asignment)[np.invert(categories)]) #number of true negatives

sum(asignment[categories]) #number of true positives 

In [ ]:
sets = [2,3,4,5]
cadds = [5,10,15,20]

for s in sets:
    
    top_genes = skat.order_by('p_value').id.take(s)   
   
    for c in cadds:
        
        variants_controls = np.zeros((len(top_genes)))
        variants_gts = np.zeros((len(top_genes)))
    
        variants_controls_test = np.zeros((len(top_genes)))
        variants_gts_test = np.zeros((len(top_genes)))

        model_asignment = np.zeros((len(top_genes), 80))
        test_asignment = np.zeros((len(top_genes), 145)) 
    
        for rows, n in enumerate(top_genes):
            
            mt_skat_log = mt_for_skat.filter_rows(mt_for_skat.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
            mt_test_skat_log = mt_test.filter_rows(mt_test.nearest_genes_20kb.contains(n)) # to ma rodziny

            mt_skat_log = mt_skat_log.filter_rows(mt_skat_log.cadd > c)
            mt_skat_log = mt_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
            non_refs = mt_skat_log.non_refs.collect()

            mt_test_skat_log = mt_test_skat_log.filter_rows(mt_test_skat_log.cadd > c)
            mt_test_skat_log = mt_test_skat_log.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log.GT.is_non_ref())) #count variants per sample, prepare also test matrix
            non_refs_test = mt_test_skat_log.non_refs.collect()

            variants_gts[rows] = np.mean(np.array(non_refs)[categories])
            variants_controls[rows] = np.mean(np.array(non_refs)[np.invert(categories)])

            variants_gts_test[rows] = np.mean(np.array(non_refs_test)[categories_test])
            variants_controls_test[rows] = np.mean(np.array(non_refs_test)[np.invert(categories_test)])

            results = (non_refs - variants_controls[rows]) 
            results_test = (non_refs_test - variants_controls[rows]) 

            model_asignment[rows] = (results)
            test_asignment[rows] = (results_test)

        model_asignment = np.sum(model_asignment, axis = 0)
        test_asignment = np.sum(test_asignment, axis = 0)
            
        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_gts'+str(s)+'cadd'+str(c), variants_gts)
        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_controls'+str(s)+'cadd'+str(c), variants_controls)

        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_gts_test'+str(s)+'cadd'+str(c), variants_gts_test)
        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-variants_controls_test'+str(s)+'cadd'+str(c), variants_controls_test)

        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-model_asignment'+str(s)+'cadd'+str(c), model_asignment)
        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-test_asignment'+str(s)+'cadd'+str(c), test_asignment)

        false_pos = []
        true_pos = []
            
        for x in np.linspace(-25,20,20000):
            false_pos.append(np.sum((test_asignment > x)[np.invert(categories_test)])/56)
            true_pos.append(np.sum((test_asignment > x)[categories_test])/89)
                
        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-false_pos'+str(s)+'cadd'+str(c), false_pos)
        np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-true_pos'+str(s)+'cadd'+str(c), true_pos)
            
        print(str(s)+'cadd'+str(c))
        print(variants_gts)
        print(variants_controls)

false_pos = []
true_pos = []

for s in sets:
        for c in cadds:
            false_pos.append(np.load('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/numpy/may-false_pos'+str(s)+'cadd'+str(c)+'.npy'))
            true_pos.append(np.load('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/results/numpy/may-true_pos'+str(s)+'cadd'+str(c)+'.npy'))

### ROC

In [ ]:
from bokeh.models import Label, LabelSet

colors = bokeh.palettes.Category20[16]

y = np.linspace(0,1,10)
x = np.linspace(0,1,10)

from bokeh.plotting import figure, output_notebook, show

output_notebook

p = figure(plot_width=800, plot_height=800)
p.line(x, y, line_width=4, line_color='lightgrey')

for i in range(16):
    p.line(false_pos[i], true_pos[i], line_width=1, alpha=0.5, line_color=colors[i])
    
p.line(false_pos[13], true_pos[13], line_width=4, line_color='orange')

p.circle(false_pos[13][10740], true_pos[13][10740], size=12, color="darkred", alpha=1)
p.circle(false_pos[13][10760], true_pos[13][10760], size=12, color="red", alpha=1)

labels = LabelSet(x=[false_pos[13][10740], false_pos[13][10760]], y=[false_pos[13][10740], false_pos[13][10760]], text=['point A', 'point B'],
              x_offset=5, y_offset=5, render_mode='canvas')

p.add_layout(labels)


p.xaxis.axis_label = 'false positives'
p.yaxis.axis_label = 'true positives'

p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"

show(p)

auc = []

for i in range(16):
    auc.append(-np.trapz(true_pos[i], false_pos[i]))

In [ ]:
max(auc)

In [ ]:
np.where(auc == max(auc))

In [ ]:
factors = []
for s in sets:
    for c in cadds:
        factors.append(str(s)+'cadds:'+str(c))
        

In [ ]:
factors[13]

Get the false positive rate of models:

In [ ]:
genes_background = mt_test.nearest_genes_20kb.collect()
genes_background_1 =  set([val for sublist in genes_background for val in sublist])

genes_background = [x for x in genes_background_1 if x in allgenes]

genes_background = np.array(genes_background)
np.save('genes_background', genes_background)

In [ ]:
genes_background = np.load('/net/archive/groups/plggneuromol/GTS-analysis/analysis/numpy/genes_background.npy')

sets = [2,3,4,5]
cadds = [5,10,15,20]

categories = mt_for_skat.category.collect()
categories_test = mt_test.category.collect()

mt_test = mt_test.filter_rows(mt_test.cadd > 5)
mt_test = checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test_5.mt')
mt_test = mt_test.filter_rows(mt_test.cadd > 10)
mt_test.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test_10.mt')
mt_test = mt_test.filter_rows(mt_test.cadd > 15)
mt_test.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test_15.mt')
mt_test = mt_test.filter_rows(mt_test.cadd > 20)
mt_test.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test_20.mt')

mt_for_skat = mt_for_skat.filter_rows(mt_for_skat.cadd > 5)
mt_for_skat.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_for_skat_5.mt')
mt_for_skat = mt_for_skat.filter_rows(mt_for_skat.cadd > 10)
mt_for_skat.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_for_skat_10.mt')
mt_for_skat = mt_for_skat.filter_rows(mt_for_skat.cadd > 15)
mt_for_skat.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_for_skat_15.mt')
mt_for_skat = mt_for_skat.filter_rows(mt_for_skat.cadd > 20)
mt_for_skat.checkpoint('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_for_skat_20.mt')

In [ ]:
for gene in range(28,31):

    randoms = np.random.randint(len(genes_background), size=5)
    top_genes = [j for i, j in enumerate(genes_background) if i in randoms]

    print(top_genes)
        
    for c_idx, c in enumerate(cadds):

        test_asignment = np.zeros((len(top_genes), 145)) 
        mt_skat_log = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_for_skat_'+str(c)+'.mt')
        mt_test_skat_log = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/mt_test_'+str(c)+'.mt')
      
        for rows, n in enumerate(top_genes):

            mt_skat_log_gene = mt_skat_log.filter_rows(mt_skat_log.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
        
            mt_test_skat_log_gene = mt_test_skat_log.filter_rows(mt_test_skat_log.nearest_genes_20kb.contains(n)) # to ma rodziny

            mt_skat_log_gene = mt_skat_log_gene.annotate_cols(non_refs = hl.agg.count_where(mt_skat_log_gene.GT.is_non_ref())) #count variants per sample of gnomads and heavy tics  
            non_refs = mt_skat_log_gene.non_refs.collect()
                                                
            mt_test_skat_log_gene = mt_test_skat_log_gene.annotate_cols(non_refs = hl.agg.count_where(mt_test_skat_log_gene.GT.is_non_ref())) #count variants per sample, prepare also test matrix
            non_refs_test = mt_test_skat_log_gene.non_refs.collect()

            variants_controls = np.mean(np.array(non_refs)[np.invert(categories)])
                
            test_asignment[rows] = (non_refs_test - variants_controls)
            
            print(test_asignment.shape)
                
        for s in sets:
        
            test_asignment_subset = test_asignment[range(5-s,5), :]
            
            print(test_asignment_subset.shape)
                
            test_asignment_subset = np.sum(test_asignment_subset, axis = 0)
                
            false_pos = []
            true_pos = []
            
            for x in np.linspace(-100,100,10000):
            
                false_pos.append(np.sum((test_asignment_subset > x)[np.invert(categories_test)])/53)
                true_pos.append(np.sum((test_asignment_subset > x)[categories_test])/91)
            
            print(np.trapz(false_pos, true_pos))
        
            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/auc-test/false_pos'+str(s)+'cadd'+str(c)+'gene'+str(gene), false_pos)
            np.save('/net/archive/groups/plggneuromol/GTS-analysis/data/auc-test/true_pos'+str(s)+'cadd'+str(c)+'gene'+str(gene), true_pos)
        
    print('I have completed iteration number: ' + str(gene))

## AUC

In [ ]:
false_pos = []
true_pos = []

for s in sets:
    for c in cadds:
        for gene in range(0,31):    
            a = np.load('/net/archive/groups/plggneuromol/GTS-analysis/data/auc-test/false_pos'+str(s)+'cadd'+str(c)+'gene'+str(gene)+'.npy')
            b = np.load('/net/archive/groups/plggneuromol/GTS-analysis/data/auc-test/true_pos'+str(s)+'cadd'+str(c)+'gene'+str(gene)+'.npy')
            
            a = a*53/56 #this is to correct the number of healthy
            b = b*91/89 #this is to correct the number with GTS

                
            if (a[0] == 1) & (a[9999] == 0) & (b[0] == 1) & (b[9999] == 0):
                false_pos.append(a)
                true_pos.append(b)

In [ ]:
aucs = []
for i in range(478):
    aucs.append(np.trapz(true_pos[i], false_pos[i]))

In [ ]:
aucs = np.array(aucs)

In [ ]:
np.percentile(-aucs, 99.999)

In [ ]:
test1 = np.array(([0,1,2,4,5], [0,2,2,3,5]))
test2 = np.array(([1,1,1,1,1], [2,2,2,2,2]))

In [ ]:
y = np.linspace(0,1,10)
x = np.linspace(0,1,10)

In [ ]:
false_pos_ori = []
true_pos_ori = []

for s in sets:
        for c in cadds:
            false_pos_ori.append(np.load('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-false_pos'+str(s)+'cadd'+str(c)+'.npy'))
            true_pos_ori.append(np.load('/net/archive/groups/plggneuromol/GTS-analysis/data/numpy/may-true_pos'+str(s)+'cadd'+str(c)+'.npy'))

In [ ]:
p4 = figure(plot_width=800, plot_height=800)
p4.line(x, y, line_width=4, line_color='lightgrey')


for i in range(0,296):
    p4.line(false_pos[i], true_pos[i], line_width=1, alpha=0.25)
    
p4.line(false_pos_ori[13], true_pos_ori[13], line_width=4, line_color='orange')

p4.xaxis.axis_label = 'false positives'
p4.yaxis.axis_label = 'true positives'


p4.xaxis.axis_label_text_font_size = "15pt"
p4.xaxis.major_label_text_font_size = "15pt"
p4.yaxis.axis_label_text_font_size = "15pt"
p4.yaxis.major_label_text_font_size = "15pt"

# show the results
show(p4)